# Практическое задание к уроку 8 часть 2

In [35]:
import pandas as pd
import pickle
import dill
dill._dill._reverse_typemap['ClassType'] = type

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score

import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization


In [36]:
X_test = pd.read_csv("X_test_lesson_8.csv")
y_test = pd.read_csv("y_test_lesson_8.csv",header=None)
X_test.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,70.0,0,582,0,40,0,51000.0,2.7,136,1,1,250
1,50.0,1,298,0,35,0,362000.0,0.9,140,1,1,240
2,45.0,0,2442,1,30,0,334000.0,1.1,139,1,0,129


In [37]:
with open('model_trained_XGB.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [38]:
predictions = pipeline.predict_proba(X_test)

In [41]:
def graf_prediction(algorithm,testing_x,testing_y,predictions, 
                             probabilities ,cf,threshold_plot):
    print ("\n Classification report : \n",classification_report(testing_y,predictions))
    print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    


In [42]:
graf_prediction(pipeline.steps[1][1],
                         X_test,y_test,pipeline.predict(X_test), pipeline.predict_proba(X_test),
                         "coefficients",threshold_plot = True)


 Classification report : 
               precision    recall  f1-score   support

           0       0.73      0.89      0.80        57
           1       0.79      0.55      0.65        42

    accuracy                           0.75        99
   macro avg       0.76      0.72      0.73        99
weighted avg       0.76      0.75      0.74        99

Accuracy   Score :  0.7474747474747475
Area under curve :  0.7211779448621555 



e:\anaconda3\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 10. Бонусный вопрос: у вас есть на выбор два варианта разделения вашего исходного датасета - на train/test и на train/test/val. Подумайте, в каких ситуациях предпочтительнее один или второй вариант либо всегда лучше какой-то один? Какой бы выбрали вы?

если выборка небольшая то на train/test при большой выборке можно train/test/val